In [6]:
!pip install --upgrade pip
!pip install  \
    torch \
    torchdata --quiet

!pip install \
    transformers==4.27.2 \ 


  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
  Using cached pip-23.3.1-py3-none-any.whl.metadata (3.5 kB)
Using cached pip-23.3.1-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
C:\Users\natalr2\AppData\Local\anaconda3\python.exe -m pip install --upgrade pip
ERROR: Directory '\\' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
pip install datasets

In [9]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [11]:
ds_hf = "knkarthick/dialogsum"

ds = load_dataset(ds_hf)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/natalr2/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [17]:
example_indices = [40,200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices): # to get a counter and the value from the iterable in a loop
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(ds['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY: ')
    print(ds['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Ex

In [23]:
# load the model using Flan t5 -- flan t5 is a general model that can do wide variety of tasks
# check pytorch running

model_name= 'google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name) #check what it does

C:\Users\natalr2\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\natalr2\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True) #convert raw test in a vector space to be processed by flan t5 model

In [26]:
# checking vector

sentence = "Rebirth of a man"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
    sentence_encoded["input_ids"][0],
    skip_special_tokens=True
)

print(sentence_encoded["input_ids"][0])

tensor([  419, 20663,    13,     3,     9,   388,     1])


In [27]:
print(sentence_decoded)

Rebirth of a man


In [30]:
for i, index in enumerate(example_indices): # using enumerate to get a counter and the value from the iterable in a loop
    dialogue = ds['test'][index]['dialogue']
    summary = ds['test'][index]['summary']
    
    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens=50,
        )[0],
        skip_special_tokens = True
    )
                       
                                             
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY: \n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING: \n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING: 
Person1: It's ten to nine.

-----------------------------

In [33]:
for i, index in enumerate(example_indices): # using enumerate to get a counter and the value from the iterable in a loop
    dialogue = ds['test'][index]['dialogue']
    summary = ds['test'][index]['summary']

    
    prompt = f"""
Summarize the following conversation

{dialogue}

Summary:
    """
    
    
    
    inputs = tokenizer(prompt, return_tensors='pt') #using prompt instead of the dialogue to test WITH prompting
    output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens=50,
        )[0],
        skip_special_tokens = True
    )
                       
                                             
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY: \n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITH PROMPT ENGINEERING: \n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT PROMPT ENGINEERING: 
T

In [37]:
for i, index in enumerate(example_indices): # using enumerate to get a counter and the value from the iterable in a loop
    dialogue = ds['test'][index]['dialogue']
    summary = ds['test'][index]['summary']

    
    prompt = f"""
Dialogue

{dialogue}

What was going on?
"""
    
    
    
    inputs = tokenizer(prompt, return_tensors='pt') #using prompt instead of the dialogue to test ZERO shot prompting
    output = tokenizer.decode(
        model.generate(
            inputs['input_ids'],
            max_new_tokens=50,
        )[0],
        skip_special_tokens = True
    )
                       
                                             
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY: \n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT PROMPT ENGINEERING: \n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT PROMPT ENGINEERING: 
Tom is late. He has to 

In [41]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    
    for index in example_indices_full: 
        dialogue = ds['test'][index]['dialogue']
        summary = ds['test'][index]['summary']

    # the stop sequene '{summar} /n/n/n/ is important for flan t5. other models my have their own preferred stop sequence
        prompt += f"""
Dialogue

{dialogue}

What was going on?
{summary}



"""
    
    dialogue = ds['test'][example_index_to_summarize]['dialogue']
    
    prompt += f"""
Dialogue:
    
{dialogue}

What was going on?
"""
    
    return prompt



In [45]:
example_indices_full = [40]

example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.




Dialogue:
    
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you als

In [46]:
summary = ds['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY: /n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT: \n{output}')
        

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: /n#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT: 
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


In [47]:
example_indices_full = [40, 80, 120]

example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.




Dialogue

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. M

In [50]:
summary = ds['test'][example_index_to_summarize]['summary'] # in this case, few shot does not help that much and anything above 5/6 shots will tipically not help either

# pay attention in a input-context-length in this case 512

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY: \n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT: \n{output}')   

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT: 
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


In [57]:
# changing parameters

#generation_config = GenerationConfig(max_new_tokens=50)
#generation_config = GenerationConfig(max_new_tokens=10) #make the output text too short, dialogue will be cut
#generation_config = GenerationConfig(max_new_tokens=50, do_sample = True, temperature = 0.1) # do_sample = True and changing temp, makes get more flexibility in the output
#generation_config = GenerationConfig(max_new_tokens=50, do_sample = True, temperature = 0.5)
generation_config = GenerationConfig(max_new_tokens=50, do_sample = True, temperature = 1.0)


inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config = generation_config, #adding the configuration to the output
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT: \n{output}') 
print(dash_line)
print(f'BASELINE HUMAN SUMMARY: \n{summary}\n') 


---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT: 
Person1 suggests that Person2 should upgrade his system. Besides, Person1 recommends adding a painting program to have a fast processor.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

